<div style="display: flex; background-color: RGB(255,114,0);" >

# PROJET - Bonheur World <mark>Exploration des données et nettoyage</mark>
</div>

<div style="display: flex; background-color: Blue; padding: 15px;" >

## 1.Mission 
</div>

<div style="display: flex; background-color: Green; padding: 7px;" >

### 1.1. Rappel sur sujet
</div>

Challenge goals

How accurately can we predict regional temperature anomalies based on past and neighbouring climate observations ?

Dans cette étude de cas, nous formerons un algorithme d'apprentissage automatique non supervisé pour regrouper les pays en fonction de caractéristiques telles que la production économique, le soutien social, l'espérance de vie, la liberté, l'absence de corruption et la générosité. Le Rapport sur le bonheur dans le monde détermine l'état du bonheur mondial. Les scores de bonheur et les données de classement ont été collectés en demandant aux individus de classer leur vie de 0 (la pire vie possible) à 10 (la meilleure vie possible).         

- Un notebook contenant les fonctions permettant le prétraitement des données ainsi que les résultats du clustering (en y incluant des représentations graphiques) ....
- Un support de présentation qui présente la démarche et les résultats du clustering.

<div style="display: flex; background-color: Green; padding: 7px;" >

### 1.2. Description du notebook
</div>

Ce note book, a pour objectif d'explorer les données et d'effectuer le nettoyage.

<div style="display: flex; background-color: Green; padding: 7px;" >

### 1.3. Description des données
</div>



|column        | Description                             |
|--------------| --------------------------------------- |
|country_official | nom officiel du pays |
|country | nom du pays reçu à l'origine|
|year | année|
|score | score du bonheur|
|PIB | PIB par habitant|
|Soutien | Soutien social|
|Esperance vie BS | Espérance de vie en Bonne Santé|
|Liberte des choix de vie | |
|Generosite | la générosité perçue|
|Corruption | le faible niveau de corruption perçue|
|Regional indicator||
|id_country | identifiant officiel du pays|
|alpha3 | code alpha3 officiel du pays|
|alpha2 | code alpha2 officiel du pays|
|continent_code | code continent officiel du pays|
|latitude | latitude du pays|
|longitude | longitude du pay|s
|Annual Sunshine | durée d'ensoleillement du pays|
|Annual Sunshine NCDC Computed||
|deaths_rural_urban | nombre de morts en zone urbaine|
|divorces | nombre de divorce|
|Precipitations in million cubic metres | volume de précipitations|
|rate of Population connected to wastewater collecting system | ratio de la population avec l'eau courante|
|rate of Population connected to wastewater treatment | ratio de la population connectée au tout à l'égout|
|gini | indice de gini|
|intentional homicide victims Female nb | nombre de femmes victimes d'homicide |
|intentional homicide victims Male nb | nombre d'hommes victimes d'homicide |
|Homicide victime nb | nombre de victimes d'homicide |


In [1]:
from os import getcwd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from tqdm import tqdm
import csv
from bonheur_bed_ara import *
from IPython.core.display import HTML
import plotly.express as px

<div style="display: flex; background-color: Blue; padding: 15px;" >

## 2.Chargement des données
</div>

In [2]:
# ---------------------------------------------------------------------------------------------
#                               MAIN
# ---------------------------------------------------------------------------------------------
verbose = False

# Récupère le répertoire du programme
file_path = getcwd() + "\\"
data_set_path = file_path + "dataset\\"
country_col_name = "country"
country_official_col_name = "country_official"
data_set_file_name = "DATA_SET_FULL.csv"

print(f"Current execution path : {file_path}")
print(f"Dataset path : {data_set_path}")

Current execution path : c:\Users\User\WORK\workspace-ia\PROJETS\projet_bonheur_bed\
Dataset path : c:\Users\User\WORK\workspace-ia\PROJETS\projet_bonheur_bed\dataset\


<div style="display: flex; background-color: Green; padding: 7px;" >

### 2.1. Chargement
</div>

Le code continent `NA` pour `North America` est interprété comme un NAN, il faut donc en tenir compte au chargement des données.

In [3]:
# Le code continent North American = NA était interprété en tant que NaN et non sa valeur
df_origin = pd.read_csv(data_set_path+data_set_file_name, quoting=csv.QUOTE_NONNUMERIC, na_values=["", np.nan], keep_default_na=False, sep=',')
df_origin = df_origin.sort_values(by=[country_official_col_name, "year"])

print(f"{df_origin.shape} données chargées ------> {list(df_origin.columns)}")
df_origin.head()

(2359, 28) données chargées ------> ['country_official', 'country', 'year', 'score', 'PIB', 'Soutien', 'Esperance vie BS', 'Liberte des choix de vie', 'Generosite', 'Corruption', 'Regional indicator', 'id_country', 'alpha3', 'alpha2', 'continent_code', 'latitude', 'longitude', 'Annual Sunshine', 'Annual Sunshine NCDC Computed', 'deaths_rural_urban', 'divorces', 'Precipitations in million cubic metres', 'rate of Population connected to wastewater collecting system', 'rate of Population connected to wastewater treatment', 'gini', 'intentional homicide victims Female nb', 'intentional homicide victims Male nb', 'Homicide victime nb']


,country_official,country,year,score,PIB,Soutien,Esperance vie BS,Liberte des choix de vie,Generosite,Corruption,...,Annual Sunshine NCDC Computed,deaths_rural_urban,divorces,Precipitations in million cubic metres,rate of Population connected to wastewater collecting system,rate of Population connected to wastewater treatment,gini,intentional homicide victims Female nb,intentional homicide victims Male nb,Homicide victime nb
443,Arab Republic of Egypt,Egypt,2005.0,5.168,9.036,0.848,59.70,0.817,NaN,NaN,...,3293.3,450646.0,65047.0,1300.0,NaN,NaN,NaN,63.0,459.0,10962.0
828,Arab Republic of Egypt,Egypt,2007.0,5.541,9.135,0.686,59.82,0.609,-0.121,NaN,...,3790.8,450596.0,77878.0,1300.0,NaN,NaN,NaN,163.0,517.0,14280.0
1082,Arab Republic of Egypt,Egypt,2008.0,4.632,9.186,0.738,59.88,NaN,-0.087,0.914,...,3790.8,461934.0,84430.0,NaN,NaN,NaN,NaN,110.0,856.0,20286.0
936,Arab Republic of Egypt,Egypt,2009.0,5.066,9.213,0.744,59.94,0.611,-0.100,0.801,...,3298.9,476592.0,141467.0,NaN,NaN,NaN,NaN,193.0,719.0,19152.0
961,Arab Republic of Egypt,Egypt,2010.0,4.669,9.244,0.769,60.00,0.486,-0.076,0.826,...,3666.4,483385.0,149376.0,NaN,NaN,NaN,NaN,197.0,1642.0,38619.0


<div style="display: flex; background-color: Green; padding: 7px;" >

### 2.2. Exploration de la forme
</div>

In [4]:
df_origin[df_origin['continent_code'].isna()]

,country_official,country,year,score,PIB,Soutien,Esperance vie BS,Liberte des choix de vie,Generosite,Corruption,...,Annual Sunshine NCDC Computed,deaths_rural_urban,divorces,Precipitations in million cubic metres,rate of Population connected to wastewater collecting system,rate of Population connected to wastewater treatment,gini,intentional homicide victims Female nb,intentional homicide victims Male nb,Homicide victime nb


In [6]:
df = df_origin.copy()

<div style="display: flex; background-color: indigo;" >

#### Encodage des continents
</div>

In [5]:
# Encodage des continents :
from sklearn.preprocessing import LabelEncoder

In [7]:
transformer_continent = LabelEncoder()
df['continent_encode'] = transformer_continent.fit_transform(df['continent_code'])
df = df[['country_official', 'country', 'continent_code','continent_encode','year', 'score', 'PIB', 'Soutien',
       'Esperance vie BS', 'Liberte des choix de vie', 'Generosite',
       'Corruption', 'Regional indicator', 'alpha3', 'alpha2', 'latitude', 'longitude', 'id_country', 'Annual Sunshine',
       'Annual Sunshine NCDC Computed', 'deaths_rural_urban', 'divorces',
       'Precipitations in million cubic metres',
       'rate of Population connected to wastewater collecting system',
       'rate of Population connected to wastewater treatment', 'gini',
       'intentional homicide victims Female nb',
       'intentional homicide victims Male nb', 'Homicide victime nb'
       ]]

<div style="display: flex; background-color: indigo;" >

#### Typage des données
</div>

In [8]:
df["year"] = df["year"].astype(int)
df["id_country"] = df["id_country"].astype(int)

In [9]:
df.dtypes

country_official                                                 object
country                                                          object
continent_code                                                   object
continent_encode                                                  int32
year                                                              int32
score                                                           float64
PIB                                                             float64
Soutien                                                         float64
Esperance vie BS                                                float64
Liberte des choix de vie                                        float64
Generosite                                                      float64
Corruption                                                      float64
Regional indicator                                               object
alpha3                                                          

In [10]:
df.describe(include="all")

,country_official,country,continent_code,continent_encode,year,score,PIB,Soutien,Esperance vie BS,Liberte des choix de vie,...,Annual Sunshine NCDC Computed,deaths_rural_urban,divorces,Precipitations in million cubic metres,rate of Population connected to wastewater collecting system,rate of Population connected to wastewater treatment,gini,intentional homicide victims Female nb,intentional homicide victims Male nb,Homicide victime nb
count,2359,2359,2359,2359.000000,2359.000000,2359.000000,2265.000000,2274.000000,2242.000000,2256.000000,...,1470.000000,1.035000e+03,9.460000e+02,5.340000e+02,461.000000,386.000000,841.000000,1063.000000,1057.000000,1.055000e+03
unique,164,190,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Arab Republic of Egypt,Egypt,AS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,17,17,707,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,1.518016,2014.507842,5.442263,8.841420,0.817168,59.332300,0.730743,...,2363.185646,3.624518e+05,7.289547e+04,7.909549e+05,66.355360,67.537196,36.906421,599.159925,2555.536424,6.635291e+04
std,NaN,NaN,NaN,1.384665,4.768411,1.121303,2.249192,0.137225,17.045576,0.151687,...,672.812090,1.180071e+06,2.678078e+05,2.366401e+06,25.828580,33.103334,7.890536,2207.075003,7445.275762,1.901684e+05
min,NaN,NaN,NaN,0.000000,2005.000000,2.375000,0.000000,0.000000,0.000000,0.000000,...,8.700000,1.216000e+03,4.200000e+01,1.540000e+01,4.900000,0.000000,24.000000,0.000000,0.000000,0.000000e+00
25%,NaN,NaN,NaN,0.000000,2011.000000,4.624500,8.167000,0.746000,56.400000,0.636000,...,1820.400000,2.878850e+04,5.162250e+03,3.251570e+04,51.040001,46.625000,30.900000,22.000000,45.000000,1.407000e+03
50%,NaN,NaN,NaN,1.000000,2015.000000,5.383000,9.370000,0.838000,64.800000,0.750500,...,2438.600000,8.087600e+04,1.310950e+04,1.010552e+05,69.800003,80.299999,35.400000,58.000000,162.000000,5.124000e+03
75%,NaN,NaN,NaN,2.000000,2019.000000,6.256500,10.263000,0.909000,68.200000,0.853000,...,2892.850000,2.579000e+05,3.583625e+04,3.659614e+05,87.000000,97.000000,42.000000,202.000000,752.000000,2.022300e+04


In [16]:
df_clean = df.copy()
df_clean = df_clean.drop(['Annual Sunshine', 'intentional homicide victims Male nb', 'intentional homicide victims Female nb'], axis=1)

In [150]:
def complete_df_with_water_data(df_clean, verbose=0):
    col_names = ['rate of Population connected to wastewater collecting system', 'rate of Population connected to wastewater treatment']
    df_completed = df_clean.copy()

    i = 0

    for col_name in col_names:
        # Création de la DF temporaire avec la dernière valeur par pays
        df_temp_max = df_completed[df_completed[col_name].notna()]
        df_temp_max = df_temp_max.sort_values(by=["country_official","year"], ascending=False)
        df_temp_max = df_temp_max[[country_official_col_name, "year", col_name]]
        if verbose>1:
            print(df_clean['year']>df_temp_max['year'])
            # print("SHAPE 1 : df_clean[df_clean['year']>df_temp_max['year']]       => ",df_clean[df_clean['year']>df_temp_max[year_col_name]].shape)
            # print("SHAPE 2 : df_temp_max[df_temp_max['year']<df_temp_max['year']] => ",df_temp_max[df_temp_max['year']<df_temp_max[year_col_name]].shape)
        year_col_name = "year_max_"+str(i)
        df_temp_max = df_temp_max.rename(columns={"year": year_col_name, col_name : col_name+"_max"})
        # on ne garde que la dernière année
        df_temp_max = df_temp_max.drop_duplicates(subset=["country_official"], keep="first")

        # merge des 2 DF pour pouvoir compléter les données
        df_completed = df_completed.merge(df_temp_max, on=country_official_col_name, how="left", indicator=False)
        df_completed.loc[(df_completed[col_name].isna()) & (df_completed["year"]>df_completed[year_col_name]), col_name] = df_completed[col_name+"_max"]

        if verbose:
            print("SHAPE df_clean : => ",df_clean.shape)
            print("SHAPE df_temp_max : => ",df_temp_max.shape)

        # Suppression des colonnes ajoutées
        df_completed = df_completed.drop(columns=[year_col_name, col_name+"_max"])
        # DF complétée
        if verbose:
            print(f"{col_name} : {df_clean[col_name].isna().sum()} NA BEFORE => {df_completed[col_name].isna().sum()} NA AFTER")
        i += 1
    return df_completed


In [152]:
df_completed = complete_df_with_water_data(df_clean, verbose=verbose)
df_completed.head()

SHAPE df_clean : =>  (2359, 26)
SHAPE df_temp_max : =>  (76, 3)
rate of Population connected to wastewater collecting system : 1898 NA BEFORE => 1279 NA AFTER
SHAPE df_clean : =>  (2359, 26)
SHAPE df_temp_max : =>  (66, 3)
rate of Population connected to wastewater treatment : 1973 NA BEFORE => 1453 NA AFTER


,country_official,country,continent_code,continent_encode,year,score,PIB,Soutien,Esperance vie BS,Liberte des choix de vie,...,longitude,id_country,Annual Sunshine NCDC Computed,deaths_rural_urban,divorces,Precipitations in million cubic metres,rate of Population connected to wastewater collecting system,rate of Population connected to wastewater treatment,gini,Homicide victime nb
0,Arab Republic of Egypt,Egypt,AF,0,2005,5.168,9.036,0.848,59.70,0.817,...,119.525419,818,3293.3,450646.0,65047.0,1300.0,NaN,NaN,NaN,10962.0
1,Arab Republic of Egypt,Egypt,AF,0,2007,5.541,9.135,0.686,59.82,0.609,...,119.525419,818,3790.8,450596.0,77878.0,1300.0,NaN,NaN,NaN,14280.0
2,Arab Republic of Egypt,Egypt,AF,0,2008,4.632,9.186,0.738,59.88,NaN,...,119.525419,818,3790.8,461934.0,84430.0,NaN,NaN,NaN,NaN,20286.0
3,Arab Republic of Egypt,Egypt,AF,0,2009,5.066,9.213,0.744,59.94,0.611,...,119.525419,818,3298.9,476592.0,141467.0,NaN,NaN,NaN,NaN,19152.0
4,Arab Republic of Egypt,Egypt,AF,0,2010,4.669,9.244,0.769,60.00,0.486,...,119.525419,818,3666.4,483385.0,149376.0,NaN,NaN,NaN,NaN,38619.0
